In [8]:
import requests
import csv
import os
from hdfs import InsecureClient

In [4]:

# Base URL for PokeAPI
base_url = "https://pokeapi.co/api/v2/ability/"

# Function to fetch data from PokeAPI
def fetch_ability_data(ability_id):
    url = f"{base_url}{ability_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        effect_entries = data.get('effect_entries', [])
        for entry in effect_entries:
            if entry['language']['name'] == 'en':
                return {
                    'id': ability_id,
                    'pokemon_ability_id': ability_id,
                    'effect': entry.get('effect', ''),
                    'language': entry['language']['name'],
                    'short_effect': entry.get('short_effect', '')
                }
    return None

# Function to save data to CSV
def save_to_csv(start_id, end_id, abilities):
    filename = f"result_{start_id}_{end_id}.csv"
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'pokemon_ability_id', 'effect', 'language', 'short_effect'])
        for ability in abilities:
            writer.writerow([ability['id'], ability['pokemon_ability_id'], ability['effect'], ability['language'], ability['short_effect']])
    return filename

# Fetch and save abilities data
def fetch_and_save_abilities(start_id, end_id):
    abilities = []
    for ability_id in range(start_id, end_id + 1):
        ability_data = fetch_ability_data(ability_id)
        if ability_data:
            abilities.append(ability_data)
    return save_to_csv(start_id, end_id, abilities)

# Fetch data and save to CSV files in chunks of 100
for i in range(0, 1000, 100):
    fetch_and_save_abilities(i + 1, i + 100)


In [17]:
# HDFS client - use 'namenode' as it is defined in docker-compose.yml
hdfs_client = InsecureClient('http://namenode:9870', user='root')

# Function to save CSV to HDFS
def save_to_hdfs(filename, hdfs_path):
    hdfs_client.upload(hdfs_path, filename)

# Function to create HDFS directories if they do not exist
def create_hdfs_directories(path):
    if not hdfs_client.status(path, strict=False):
        hdfs_client.makedirs(path)
        
# Function to save all CSV files to HDFS
def save_all_csv_to_hdfs(directory, hdfs_base_path):
    create_hdfs_directories(hdfs_base_path)
    for filename in os.listdir(directory):
        if filename.startswith("result_") and filename.endswith(".csv"):
            hdfs_path = f"{hdfs_base_path}/{filename}"
            save_to_hdfs(os.path.join(directory, filename), hdfs_path)


In [19]:
# Save all generated CSV files to HDFS
save_all_csv_to_hdfs('.', '/user/root/data_engineer_digitalskola')

ConnectionError: HTTPConnectionPool(host='namenode', port=9870): Max retries exceeded with url: /webhdfs/v1/user/root/data_engineer_digitalskola?user.name=root&op=GETFILESTATUS (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x0000020347E6BE10>: Failed to resolve 'namenode' ([Errno 11001] getaddrinfo failed)"))